In [5]:
import os

import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
from torch.optim import lr_scheduler
from torch.autograd import Variable

from preprocessing_labels import training_data,validation_data,idx_to_name,idx_to_umls,umls_to_idx

from preprocessing_images import Data,preprocessing

In [7]:
def train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs=25):
    use_gpu = torch.cuda.is_available()

    dataset_sizes = {'train': len(dataloaders['train'].dataset), 
                     'valid': len(dataloaders['valid'].dataset)}

    for epoch in range(num_epochs):
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                #breakpoint()
                model.train(False)

            running_loss = 0.0
            batch_count=0;
            for inputs, labels in dataloaders[phase]:
                
                batch_count+=1;
                #breakpoint()
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.float().cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels.float())

                optimizer.zero_grad()

                
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                
                running_loss += loss.item()
                #running_corrects += torch.sum(preds == labels.data)
                #breakpoint()
                print('epoch: ' +str(epoch+1)+'/'+str(num_epochs)+'    batch: ' + 
                      str(batch_count)+'/'+str(dataloaders[phase].__len__())+
                      '\t Loss: ' + str(loss.detach().item()));      
                
            if phase == 'train':
                train_epoch_loss = running_loss / dataset_sizes[phase]
            else:
                valid_epoch_loss = running_loss / dataset_sizes[phase]
                

        print('Epoch [{}/{}] train loss: {:.4f} ' 
              'valid loss: {:.4f}'.format(
                epoch+1, num_epochs,
                train_epoch_loss, 
                valid_epoch_loss))

    return model

In [41]:
def get_outputs(model,net_name,directory,data):
    '''saves and returns the outputs from the model after feeding in the dataset'''
    
    all_outputs=np.zeros((len(data),1000))
    
    model.eval()
    
    for t in range(len(all_outputs)):
            
            print(str(t)+'/'+str(len(data)))
        
            fullname = directory+idx_to_name[t]+'.jpg'
    
            image = Image.open(fullname).convert('RGB')
    
            image = preprocessing(image)
            
            image = Variable(image.cuda()).view((1,3,224,224))
            
            output=model(image.view(1,3,224,224))[0]
            
            output_np=output.cpu().detach().numpy()
            
            #indices_threshold=np.argwhere(output_np>threshold)
            
            all_outputs[t]=output_np
            
    np.save('models/'+net_name+'_all_outputs',all_outputs)
    
    return all_outputs

In [45]:
def create_text_files(thresholds, max_per_sample,net_name,array_of_outputs):
    '''creates the formatted submission text files for each threshold
    thresholds: list of thresholds on the interval [0,1]
    max_per_sample: the maximum amount of concepts allowed per sample
    '''
    
    filenames=[net_name+'_t'+ str(thresholds[i])+'.txt' for i in range(len(thresholds))]
    
    over_100=[]
    
    for j,threshold in enumerate(thresholds):

        array_of_outputs_sorted=-np.sort(-array_of_outputs,1)[:,:max_per_sample]
        array_of_outputs_argsorted=np.argsort(-array_of_outputs,1)[:,:max_per_sample]
        
        binary=array_of_outputs_sorted>threshold
        
        f=open(filenames[j], 'w')
        over_100_true=False
        
        for t in range(len(array_of_outputs)):
            

            
            #indices_threshold=np.argwhere(array_of_outputs>threshold)
            if t%500==0:
                pass;#print('t is ' +str(t))
            
            indices=np.argwhere(binary[t]==1)
            indices.shape=len(indices)
            
            if len(indices)>100:
                over_100_true=True
                over_100.append(t)
            
            line=idx_to_name[t]+'\t'
            for i in indices:
                line+=idx_to_umls[array_of_outputs_argsorted[t,i]]+','
                
            line=line[:-1]
            line+='\n'
            f.write(line)
        f.close()
        
        if over_100_true==True:
            print('there is at least one sample with over 100 UMLS codes')
    return

In [33]:
def load_model(model_path,resnet):
    
    if os.path.isfile(model_path):
        print("=> loading checkpoint '{}'".format(model_path))
        checkpoint = torch.load(model_path)
        resnet.load_state_dict(checkpoint['state_dict'])
        resnet.load_state_dict(checkpoint['optimizer'])
        
        
    
        if use_gpu:
            resnet = resnet.cuda()
        
        
        
        print("=> loaded checkpoint '{}' "
                  .format(model_path))
    else:
        print("=> no checkpoint found at '{}'".format(model_path))

    return resnet


def save_model(filename,net,save_path):

    state = {'state_dict': net.state_dict(),
             'optimizer': net.state_dict() }
    torch.save(state, save_path+filename)

In [22]:
use_gpu = torch.cuda.is_available()

training_path='training_path'
validation_path='validation_path'





In [23]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

preprocessing = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [24]:
training_ds=Data(training_data,directory=training_path,transform=preprocessing)

training_dl=DataLoader(training_ds,batch_size=64,shuffle=True, num_workers=0)



#validation_ds=Data(training_data, directory=validation_path,transform=preprocessing)

validation_ds=Data(validation_data, directory=validation_path,transform=preprocessing)

validation_dl=DataLoader(validation_ds,batch_size=32,shuffle=True, num_workers=0)
        
        
        

In [25]:
from resnet_new_class import resnet50
resnet = resnet50(pretrained=True)



if use_gpu:
    resnet = resnet.cuda()
    #inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())   
else:
    inputs, labels = Variable(inputs), Variable(labels)



In [26]:
        #criterion = torch.nn.BCEWithLogitsLoss()
        criterion = torch.nn.BCELoss()
        #criterion = torch.nn.MultiLabelSoftMarginLoss()
        optimizer = torch.optim.SGD(resnet.fc.parameters(), lr=.001, momentum=0.9)
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
            
        dloaders = {'train':training_dl, 'valid':validation_dl}

In [28]:
model = train_model(dloaders, resnet, criterion, optimizer, exp_lr_scheduler, num_epochs=1)

epoch: 1/1    batch: 1/885	 Loss: 0.5978981852531433
epoch: 1/1    batch: 2/885	 Loss: 0.5987117886543274
epoch: 1/1    batch: 3/885	 Loss: 0.5989238619804382
epoch: 1/1    batch: 4/885	 Loss: 0.6016965508460999
epoch: 1/1    batch: 5/885	 Loss: 0.594357967376709
epoch: 1/1    batch: 6/885	 Loss: 0.589401125907898
epoch: 1/1    batch: 7/885	 Loss: 0.5908058881759644
epoch: 1/1    batch: 8/885	 Loss: 0.5881772637367249
epoch: 1/1    batch: 9/885	 Loss: 0.5997465252876282
epoch: 1/1    batch: 10/885	 Loss: 0.5938534140586853
epoch: 1/1    batch: 11/885	 Loss: 0.5896262526512146
epoch: 1/1    batch: 12/885	 Loss: 0.5954996943473816
epoch: 1/1    batch: 13/885	 Loss: 0.5951154828071594
epoch: 1/1    batch: 14/885	 Loss: 0.5902726650238037
epoch: 1/1    batch: 15/885	 Loss: 0.5930667519569397
epoch: 1/1    batch: 16/885	 Loss: 0.5787330269813538
epoch: 1/1    batch: 17/885	 Loss: 0.5933210253715515
epoch: 1/1    batch: 18/885	 Loss: 0.602422833442688
epoch: 1/1    batch: 19/885	 Loss: 0.592

epoch: 1/1    batch: 152/885	 Loss: 0.5282263159751892
epoch: 1/1    batch: 153/885	 Loss: 0.5244697332382202
epoch: 1/1    batch: 154/885	 Loss: 0.5191242694854736
epoch: 1/1    batch: 155/885	 Loss: 0.5344805717468262
epoch: 1/1    batch: 156/885	 Loss: 0.525123655796051
epoch: 1/1    batch: 157/885	 Loss: 0.5268009305000305
epoch: 1/1    batch: 158/885	 Loss: 0.5234742760658264
epoch: 1/1    batch: 159/885	 Loss: 0.5173026323318481
epoch: 1/1    batch: 160/885	 Loss: 0.5233820676803589
epoch: 1/1    batch: 161/885	 Loss: 0.5297317504882812
epoch: 1/1    batch: 162/885	 Loss: 0.5248143076896667
epoch: 1/1    batch: 163/885	 Loss: 0.5298288464546204
epoch: 1/1    batch: 164/885	 Loss: 0.5178138017654419
epoch: 1/1    batch: 165/885	 Loss: 0.5184235572814941
epoch: 1/1    batch: 166/885	 Loss: 0.5270469188690186
epoch: 1/1    batch: 167/885	 Loss: 0.5071637034416199
epoch: 1/1    batch: 168/885	 Loss: 0.5217418074607849
epoch: 1/1    batch: 169/885	 Loss: 0.5292231440544128
epoch: 1/1 

epoch: 1/1    batch: 300/885	 Loss: 0.4785628318786621
epoch: 1/1    batch: 301/885	 Loss: 0.4681975245475769
epoch: 1/1    batch: 302/885	 Loss: 0.462735652923584
epoch: 1/1    batch: 303/885	 Loss: 0.4740357995033264
epoch: 1/1    batch: 304/885	 Loss: 0.47132012248039246
epoch: 1/1    batch: 305/885	 Loss: 0.47444868087768555
epoch: 1/1    batch: 306/885	 Loss: 0.471814900636673
epoch: 1/1    batch: 307/885	 Loss: 0.4690101444721222
epoch: 1/1    batch: 308/885	 Loss: 0.46435248851776123
epoch: 1/1    batch: 309/885	 Loss: 0.4716828465461731
epoch: 1/1    batch: 310/885	 Loss: 0.4691104590892792
epoch: 1/1    batch: 311/885	 Loss: 0.4754882752895355
epoch: 1/1    batch: 312/885	 Loss: 0.4663926064968109
epoch: 1/1    batch: 313/885	 Loss: 0.4640657901763916
epoch: 1/1    batch: 314/885	 Loss: 0.4593060314655304
epoch: 1/1    batch: 315/885	 Loss: 0.46410420536994934
epoch: 1/1    batch: 316/885	 Loss: 0.458583265542984
epoch: 1/1    batch: 317/885	 Loss: 0.4607135057449341
epoch: 1/

epoch: 1/1    batch: 448/885	 Loss: 0.4184923768043518
epoch: 1/1    batch: 449/885	 Loss: 0.4257417619228363
epoch: 1/1    batch: 450/885	 Loss: 0.4301098585128784
epoch: 1/1    batch: 451/885	 Loss: 0.41372552514076233
epoch: 1/1    batch: 452/885	 Loss: 0.4242723286151886
epoch: 1/1    batch: 453/885	 Loss: 0.4226642847061157
epoch: 1/1    batch: 454/885	 Loss: 0.417049378156662
epoch: 1/1    batch: 455/885	 Loss: 0.4211217164993286
epoch: 1/1    batch: 456/885	 Loss: 0.4126836955547333
epoch: 1/1    batch: 457/885	 Loss: 0.4101419746875763
epoch: 1/1    batch: 458/885	 Loss: 0.4208027124404907
epoch: 1/1    batch: 459/885	 Loss: 0.4174174964427948
epoch: 1/1    batch: 460/885	 Loss: 0.429330438375473
epoch: 1/1    batch: 461/885	 Loss: 0.4253551661968231
epoch: 1/1    batch: 462/885	 Loss: 0.4183047413825989
epoch: 1/1    batch: 463/885	 Loss: 0.4156930446624756
epoch: 1/1    batch: 464/885	 Loss: 0.4111558496952057
epoch: 1/1    batch: 465/885	 Loss: 0.42289087176322937
epoch: 1/1

epoch: 1/1    batch: 596/885	 Loss: 0.37953898310661316
epoch: 1/1    batch: 597/885	 Loss: 0.37937402725219727
epoch: 1/1    batch: 598/885	 Loss: 0.3881976306438446
epoch: 1/1    batch: 599/885	 Loss: 0.38758328557014465
epoch: 1/1    batch: 600/885	 Loss: 0.38688093423843384
epoch: 1/1    batch: 601/885	 Loss: 0.385902464389801
epoch: 1/1    batch: 602/885	 Loss: 0.37911656498908997
epoch: 1/1    batch: 603/885	 Loss: 0.38710418343544006
epoch: 1/1    batch: 604/885	 Loss: 0.3835480213165283
epoch: 1/1    batch: 605/885	 Loss: 0.37937965989112854
epoch: 1/1    batch: 606/885	 Loss: 0.3788518011569977
epoch: 1/1    batch: 607/885	 Loss: 0.3746395409107208
epoch: 1/1    batch: 608/885	 Loss: 0.3654462993144989
epoch: 1/1    batch: 609/885	 Loss: 0.3762677013874054
epoch: 1/1    batch: 610/885	 Loss: 0.37341850996017456
epoch: 1/1    batch: 611/885	 Loss: 0.38018345832824707
epoch: 1/1    batch: 612/885	 Loss: 0.3868056833744049
epoch: 1/1    batch: 613/885	 Loss: 0.39117431640625
epoc

epoch: 1/1    batch: 743/885	 Loss: 0.34767961502075195
epoch: 1/1    batch: 744/885	 Loss: 0.3524033725261688
epoch: 1/1    batch: 745/885	 Loss: 0.3540012538433075
epoch: 1/1    batch: 746/885	 Loss: 0.35284385085105896
epoch: 1/1    batch: 747/885	 Loss: 0.3518555164337158
epoch: 1/1    batch: 748/885	 Loss: 0.35763606429100037
epoch: 1/1    batch: 749/885	 Loss: 0.3517318665981293
epoch: 1/1    batch: 750/885	 Loss: 0.3530275821685791
epoch: 1/1    batch: 751/885	 Loss: 0.3476139008998871
epoch: 1/1    batch: 752/885	 Loss: 0.3527000844478607
epoch: 1/1    batch: 753/885	 Loss: 0.3460646867752075
epoch: 1/1    batch: 754/885	 Loss: 0.34274980425834656
epoch: 1/1    batch: 755/885	 Loss: 0.3506351709365845
epoch: 1/1    batch: 756/885	 Loss: 0.34840866923332214
epoch: 1/1    batch: 757/885	 Loss: 0.3438209295272827
epoch: 1/1    batch: 758/885	 Loss: 0.3461146056652069
epoch: 1/1    batch: 759/885	 Loss: 0.3442099094390869
epoch: 1/1    batch: 760/885	 Loss: 0.3426194190979004
epoch

epoch: 1/1    batch: 6/443	 Loss: 0.3307766318321228
epoch: 1/1    batch: 7/443	 Loss: 0.3276350796222687
epoch: 1/1    batch: 8/443	 Loss: 0.33829939365386963
epoch: 1/1    batch: 9/443	 Loss: 0.3218573033809662
epoch: 1/1    batch: 10/443	 Loss: 0.34483644366264343
epoch: 1/1    batch: 11/443	 Loss: 0.33394765853881836
epoch: 1/1    batch: 12/443	 Loss: 0.31989458203315735
epoch: 1/1    batch: 13/443	 Loss: 0.32696807384490967
epoch: 1/1    batch: 14/443	 Loss: 0.3248728811740875
epoch: 1/1    batch: 15/443	 Loss: 0.32684624195098877
epoch: 1/1    batch: 16/443	 Loss: 0.32951825857162476
epoch: 1/1    batch: 17/443	 Loss: 0.33629804849624634
epoch: 1/1    batch: 18/443	 Loss: 0.327842652797699
epoch: 1/1    batch: 19/443	 Loss: 0.3258894085884094
epoch: 1/1    batch: 20/443	 Loss: 0.33656060695648193
epoch: 1/1    batch: 21/443	 Loss: 0.32522425055503845
epoch: 1/1    batch: 22/443	 Loss: 0.3384737968444824
epoch: 1/1    batch: 23/443	 Loss: 0.3330005407333374
epoch: 1/1    batch: 24

epoch: 1/1    batch: 155/443	 Loss: 0.31616583466529846
epoch: 1/1    batch: 156/443	 Loss: 0.3375588059425354
epoch: 1/1    batch: 157/443	 Loss: 0.3223014175891876
epoch: 1/1    batch: 158/443	 Loss: 0.33885490894317627
epoch: 1/1    batch: 159/443	 Loss: 0.3399328291416168
epoch: 1/1    batch: 160/443	 Loss: 0.31640172004699707
epoch: 1/1    batch: 161/443	 Loss: 0.3325049877166748
epoch: 1/1    batch: 162/443	 Loss: 0.35655760765075684
epoch: 1/1    batch: 163/443	 Loss: 0.3167714476585388
epoch: 1/1    batch: 164/443	 Loss: 0.32223382592201233
epoch: 1/1    batch: 165/443	 Loss: 0.31864720582962036
epoch: 1/1    batch: 166/443	 Loss: 0.350980281829834
epoch: 1/1    batch: 167/443	 Loss: 0.32376110553741455
epoch: 1/1    batch: 168/443	 Loss: 0.3346615433692932
epoch: 1/1    batch: 169/443	 Loss: 0.33040764927864075
epoch: 1/1    batch: 170/443	 Loss: 0.31993818283081055
epoch: 1/1    batch: 171/443	 Loss: 0.3400352895259857
epoch: 1/1    batch: 172/443	 Loss: 0.32539936900138855
e

epoch: 1/1    batch: 303/443	 Loss: 0.3154502809047699
epoch: 1/1    batch: 304/443	 Loss: 0.3406512439250946
epoch: 1/1    batch: 305/443	 Loss: 0.3501017391681671
epoch: 1/1    batch: 306/443	 Loss: 0.33711984753608704
epoch: 1/1    batch: 307/443	 Loss: 0.3357440233230591
epoch: 1/1    batch: 308/443	 Loss: 0.32424646615982056
epoch: 1/1    batch: 309/443	 Loss: 0.32263147830963135
epoch: 1/1    batch: 310/443	 Loss: 0.343818724155426
epoch: 1/1    batch: 311/443	 Loss: 0.33585816621780396
epoch: 1/1    batch: 312/443	 Loss: 0.33097782731056213
epoch: 1/1    batch: 313/443	 Loss: 0.3440864384174347
epoch: 1/1    batch: 314/443	 Loss: 0.33174699544906616
epoch: 1/1    batch: 315/443	 Loss: 0.3425830900669098
epoch: 1/1    batch: 316/443	 Loss: 0.3177662491798401
epoch: 1/1    batch: 317/443	 Loss: 0.3316040337085724
epoch: 1/1    batch: 318/443	 Loss: 0.3456692397594452
epoch: 1/1    batch: 319/443	 Loss: 0.33746612071990967
epoch: 1/1    batch: 320/443	 Loss: 0.3191264569759369
epoc

In [40]:

net_name='model_1'
save_path='save_path'

In [38]:
save_model(net_name,model,save_path=save_path)

In [29]:
thresholds=[.3,.5,.75,.9]

In [42]:
all_outputs=get_outputs(model,net_name,training_path,training_data)

0/56629
1/56629
2/56629
3/56629
4/56629
5/56629
6/56629
7/56629
8/56629
9/56629
10/56629
11/56629
12/56629
13/56629
14/56629
15/56629
16/56629
17/56629
18/56629
19/56629
20/56629
21/56629
22/56629
23/56629
24/56629
25/56629
26/56629
27/56629
28/56629
29/56629
30/56629
31/56629
32/56629
33/56629
34/56629
35/56629
36/56629
37/56629
38/56629
39/56629
40/56629
41/56629
42/56629
43/56629
44/56629
45/56629
46/56629
47/56629
48/56629
49/56629
50/56629
51/56629
52/56629
53/56629
54/56629
55/56629
56/56629
57/56629
58/56629
59/56629
60/56629
61/56629
62/56629
63/56629
64/56629
65/56629
66/56629
67/56629
68/56629
69/56629
70/56629
71/56629
72/56629
73/56629
74/56629
75/56629
76/56629
77/56629
78/56629
79/56629
80/56629
81/56629
82/56629
83/56629
84/56629
85/56629
86/56629
87/56629
88/56629
89/56629
90/56629
91/56629
92/56629
93/56629
94/56629
95/56629
96/56629
97/56629
98/56629
99/56629
100/56629
101/56629
102/56629
103/56629
104/56629
105/56629
106/56629
107/56629
108/56629
109/56629
110/56629


830/56629
831/56629
832/56629
833/56629
834/56629
835/56629
836/56629
837/56629
838/56629
839/56629
840/56629
841/56629
842/56629
843/56629
844/56629
845/56629
846/56629
847/56629
848/56629
849/56629
850/56629
851/56629
852/56629
853/56629
854/56629
855/56629
856/56629
857/56629
858/56629
859/56629
860/56629
861/56629
862/56629
863/56629
864/56629
865/56629
866/56629
867/56629
868/56629
869/56629
870/56629
871/56629
872/56629
873/56629
874/56629
875/56629
876/56629
877/56629
878/56629
879/56629
880/56629
881/56629
882/56629
883/56629
884/56629
885/56629
886/56629
887/56629
888/56629
889/56629
890/56629
891/56629
892/56629
893/56629
894/56629
895/56629
896/56629
897/56629
898/56629
899/56629
900/56629
901/56629
902/56629
903/56629
904/56629
905/56629
906/56629
907/56629
908/56629
909/56629
910/56629
911/56629
912/56629
913/56629
914/56629
915/56629
916/56629
917/56629
918/56629
919/56629
920/56629
921/56629
922/56629
923/56629
924/56629
925/56629
926/56629
927/56629
928/56629
929/56629


1590/56629
1591/56629
1592/56629
1593/56629
1594/56629
1595/56629
1596/56629
1597/56629
1598/56629
1599/56629
1600/56629
1601/56629
1602/56629
1603/56629
1604/56629
1605/56629
1606/56629
1607/56629
1608/56629
1609/56629
1610/56629
1611/56629
1612/56629
1613/56629
1614/56629
1615/56629
1616/56629
1617/56629
1618/56629
1619/56629
1620/56629
1621/56629
1622/56629
1623/56629
1624/56629
1625/56629
1626/56629
1627/56629
1628/56629
1629/56629
1630/56629
1631/56629
1632/56629
1633/56629
1634/56629
1635/56629
1636/56629
1637/56629
1638/56629
1639/56629
1640/56629
1641/56629
1642/56629
1643/56629
1644/56629
1645/56629
1646/56629
1647/56629
1648/56629
1649/56629
1650/56629
1651/56629
1652/56629
1653/56629
1654/56629
1655/56629
1656/56629
1657/56629
1658/56629
1659/56629
1660/56629
1661/56629
1662/56629
1663/56629
1664/56629
1665/56629
1666/56629
1667/56629
1668/56629
1669/56629
1670/56629
1671/56629
1672/56629
1673/56629
1674/56629
1675/56629
1676/56629
1677/56629
1678/56629
1679/56629
1680/56629

2334/56629
2335/56629
2336/56629
2337/56629
2338/56629
2339/56629
2340/56629
2341/56629
2342/56629
2343/56629
2344/56629
2345/56629
2346/56629
2347/56629
2348/56629
2349/56629
2350/56629
2351/56629
2352/56629
2353/56629
2354/56629
2355/56629
2356/56629
2357/56629
2358/56629
2359/56629
2360/56629
2361/56629
2362/56629
2363/56629
2364/56629
2365/56629
2366/56629
2367/56629
2368/56629
2369/56629
2370/56629
2371/56629
2372/56629
2373/56629
2374/56629
2375/56629
2376/56629
2377/56629
2378/56629
2379/56629
2380/56629
2381/56629
2382/56629
2383/56629
2384/56629
2385/56629
2386/56629
2387/56629
2388/56629
2389/56629
2390/56629
2391/56629
2392/56629
2393/56629
2394/56629
2395/56629
2396/56629
2397/56629
2398/56629
2399/56629
2400/56629
2401/56629
2402/56629
2403/56629
2404/56629
2405/56629
2406/56629
2407/56629
2408/56629
2409/56629
2410/56629
2411/56629
2412/56629
2413/56629
2414/56629
2415/56629
2416/56629
2417/56629
2418/56629
2419/56629
2420/56629
2421/56629
2422/56629
2423/56629
2424/56629

3078/56629
3079/56629
3080/56629
3081/56629
3082/56629
3083/56629
3084/56629
3085/56629
3086/56629
3087/56629
3088/56629
3089/56629
3090/56629
3091/56629
3092/56629
3093/56629
3094/56629
3095/56629
3096/56629
3097/56629
3098/56629
3099/56629
3100/56629
3101/56629
3102/56629
3103/56629
3104/56629
3105/56629
3106/56629
3107/56629
3108/56629
3109/56629
3110/56629
3111/56629
3112/56629
3113/56629
3114/56629
3115/56629
3116/56629
3117/56629
3118/56629
3119/56629
3120/56629
3121/56629
3122/56629
3123/56629
3124/56629
3125/56629
3126/56629
3127/56629
3128/56629
3129/56629
3130/56629
3131/56629
3132/56629
3133/56629
3134/56629
3135/56629
3136/56629
3137/56629
3138/56629
3139/56629
3140/56629
3141/56629
3142/56629
3143/56629
3144/56629
3145/56629
3146/56629
3147/56629
3148/56629
3149/56629
3150/56629
3151/56629
3152/56629
3153/56629
3154/56629
3155/56629
3156/56629
3157/56629
3158/56629
3159/56629
3160/56629
3161/56629
3162/56629
3163/56629
3164/56629
3165/56629
3166/56629
3167/56629
3168/56629

...

53441/56629
53442/56629
53443/56629
53444/56629
53445/56629
53446/56629
53447/56629
53448/56629
53449/56629
53450/56629
53451/56629
53452/56629
53453/56629
53454/56629
53455/56629
53456/56629
53457/56629
53458/56629
53459/56629
53460/56629
53461/56629
53462/56629
53463/56629
53464/56629
53465/56629
53466/56629
53467/56629
53468/56629
53469/56629
53470/56629
53471/56629
53472/56629
53473/56629
53474/56629
53475/56629
53476/56629
53477/56629
53478/56629
53479/56629
53480/56629
53481/56629
53482/56629
53483/56629
53484/56629
53485/56629
53486/56629
53487/56629
53488/56629
53489/56629
53490/56629
53491/56629
53492/56629
53493/56629
53494/56629
53495/56629
53496/56629
53497/56629
53498/56629
53499/56629
53500/56629
53501/56629
53502/56629
53503/56629
53504/56629
53505/56629
53506/56629
53507/56629
53508/56629
53509/56629
53510/56629
53511/56629
53512/56629
53513/56629
53514/56629
53515/56629
53516/56629
53517/56629
53518/56629
53519/56629
53520/56629
53521/56629
53522/56629
53523/56629
5352

54123/56629
54124/56629
54125/56629
54126/56629
54127/56629
54128/56629
54129/56629
54130/56629
54131/56629
54132/56629
54133/56629
54134/56629
54135/56629
54136/56629
54137/56629
54138/56629
54139/56629
54140/56629
54141/56629
54142/56629
54143/56629
54144/56629
54145/56629
54146/56629
54147/56629
54148/56629
54149/56629
54150/56629
54151/56629
54152/56629
54153/56629
54154/56629
54155/56629
54156/56629
54157/56629
54158/56629
54159/56629
54160/56629
54161/56629
54162/56629
54163/56629
54164/56629
54165/56629
54166/56629
54167/56629
54168/56629
54169/56629
54170/56629
54171/56629
54172/56629
54173/56629
54174/56629
54175/56629
54176/56629
54177/56629
54178/56629
54179/56629
54180/56629
54181/56629
54182/56629
54183/56629
54184/56629
54185/56629
54186/56629
54187/56629
54188/56629
54189/56629
54190/56629
54191/56629
54192/56629
54193/56629
54194/56629
54195/56629
54196/56629
54197/56629
54198/56629
54199/56629
54200/56629
54201/56629
54202/56629
54203/56629
54204/56629
54205/56629
5420

54805/56629
54806/56629
54807/56629
54808/56629
54809/56629
54810/56629
54811/56629
54812/56629
54813/56629
54814/56629
54815/56629
54816/56629
54817/56629
54818/56629
54819/56629
54820/56629
54821/56629
54822/56629
54823/56629
54824/56629
54825/56629
54826/56629
54827/56629
54828/56629
54829/56629
54830/56629
54831/56629
54832/56629
54833/56629
54834/56629
54835/56629
54836/56629
54837/56629
54838/56629
54839/56629
54840/56629
54841/56629
54842/56629
54843/56629
54844/56629
54845/56629
54846/56629
54847/56629
54848/56629
54849/56629
54850/56629
54851/56629
54852/56629
54853/56629
54854/56629
54855/56629
54856/56629
54857/56629
54858/56629
54859/56629
54860/56629
54861/56629
54862/56629
54863/56629
54864/56629
54865/56629
54866/56629
54867/56629
54868/56629
54869/56629
54870/56629
54871/56629
54872/56629
54873/56629
54874/56629
54875/56629
54876/56629
54877/56629
54878/56629
54879/56629
54880/56629
54881/56629
54882/56629
54883/56629
54884/56629
54885/56629
54886/56629
54887/56629
5488

55487/56629
55488/56629
55489/56629
55490/56629
55491/56629
55492/56629
55493/56629
55494/56629
55495/56629
55496/56629
55497/56629
55498/56629
55499/56629
55500/56629
55501/56629
55502/56629
55503/56629
55504/56629
55505/56629
55506/56629
55507/56629
55508/56629
55509/56629
55510/56629
55511/56629
55512/56629
55513/56629
55514/56629
55515/56629
55516/56629
55517/56629
55518/56629
55519/56629
55520/56629
55521/56629
55522/56629
55523/56629
55524/56629
55525/56629
55526/56629
55527/56629
55528/56629
55529/56629
55530/56629
55531/56629
55532/56629
55533/56629
55534/56629
55535/56629
55536/56629
55537/56629
55538/56629
55539/56629
55540/56629
55541/56629
55542/56629
55543/56629
55544/56629
55545/56629
55546/56629
55547/56629
55548/56629
55549/56629
55550/56629
55551/56629
55552/56629
55553/56629
55554/56629
55555/56629
55556/56629
55557/56629
55558/56629
55559/56629
55560/56629
55561/56629
55562/56629
55563/56629
55564/56629
55565/56629
55566/56629
55567/56629
55568/56629
55569/56629
5557

56169/56629
56170/56629
56171/56629
56172/56629
56173/56629
56174/56629
56175/56629
56176/56629
56177/56629
56178/56629
56179/56629
56180/56629
56181/56629
56182/56629
56183/56629
56184/56629
56185/56629
56186/56629
56187/56629
56188/56629
56189/56629
56190/56629
56191/56629
56192/56629
56193/56629
56194/56629
56195/56629
56196/56629
56197/56629
56198/56629
56199/56629
56200/56629
56201/56629
56202/56629
56203/56629
56204/56629
56205/56629
56206/56629
56207/56629
56208/56629
56209/56629
56210/56629
56211/56629
56212/56629
56213/56629
56214/56629
56215/56629
56216/56629
56217/56629
56218/56629
56219/56629
56220/56629
56221/56629
56222/56629
56223/56629
56224/56629
56225/56629
56226/56629
56227/56629
56228/56629
56229/56629
56230/56629
56231/56629
56232/56629
56233/56629
56234/56629
56235/56629
56236/56629
56237/56629
56238/56629
56239/56629
56240/56629
56241/56629
56242/56629
56243/56629
56244/56629
56245/56629
56246/56629
56247/56629
56248/56629
56249/56629
56250/56629
56251/56629
5625

In [46]:
create_text_files(thresholds,25,net_name,all_outputs)

In [48]:
from evaluate_f1 import main

the main function takes in a candidate file, and the ground truth file. It compares the two and outputs the mean f1 score.

In [ ]:
#main(candidate_file.txt, ground_truth.txt)